# 结构化数据：NumPy的结构化数组

大多数时候，我们的数据可以通过一个异构类型值组成的数组表示，但有时却并非如此。  
本节介绍 `NumPy` 的结构化数组和记录数组，它们为复合的、异构的数据提供了非常有效的存储。  
尽管这里列举的模式对于简单的操作非常有用，但是这些场景通常也可以用 `Pandas` 的 `DataFrame` 来实现。

In [1]:
import numpy as np

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
  img:nth-child(1) {width:30%; height: 30%;}
</style>

In [3]:
name = ['Alice', 'Bob', 'Cathy', 'Doug']
age = [25, 45, 37, 19]
weight = [55.0, 85.5, 68.0, 61.5]

这种方法有点笨，因为并没有任何信息告诉我们这三个数组是相关联的。  
如果可以用一种单一结构来存储所有的数据，那么看起来会更自然。  
`NumPy` 可以用结构化数组实现这种存储，这些结构化数组是复合数据类型的。

In [4]:
x = np.zeros(4, dtype=int)  # 生成一个简单的数组

In [5]:
data = np.zeros(4, dtype={'names':('name', 'age', 'weight'),
                          'formats':('U10', 'i4', 'f8')})
print(data.dtype)

# U10 表示“长度不超过 10 的 Unicode 字符串”，i4 表示“4 字节（即32 比特）整型”，f8 表示“8 字节（即 64 比特）浮点型”

[('name', '<U10'), ('age', '<i4'), ('weight', '<f8')]


In [6]:
data['name'] = name
data['age'] = age
data['weight'] = weight
print(data)

[('Alice', 25, 55. ) ('Bob', 45, 85.5) ('Cathy', 37, 68. )
 ('Doug', 19, 61.5)]


正如我们希望的，所有的数据被安排在一个内存块中。  
结构化数组的方便之处在于，你可以通过索引或名称查看相应的值。

In [7]:
data['name']

array(['Alice', 'Bob', 'Cathy', 'Doug'], dtype='<U10')

In [8]:
data[0]

('Alice', 25, 55.)

In [9]:
data[-1]['name']

'Doug'

In [10]:
data[data['age'] < 30]['name']

array(['Alice', 'Doug'], dtype='<U10')

请注意，如果你希望实现比上面更复杂的操作，那么你应该考虑使用 `Pandas` 包，我们将在下一章中详细介绍它。  
正如你将会看到的，`Pandas` 提供了一个 `DataFrame` 对象，该结构是构建于 `NumPy` 数组之上的，提供了很多有用的数据操作功能，其中有些与前面介绍的类似，当然也有更多没提过并且非常实用的功能。

## 1. 生成结构化数组

In [11]:
# 字典法
np.dtype({'names':('name', 'age', 'weight'),
          'formats':('U10', 'i4', 'f8')})

dtype([('name', '<U10'), ('age', '<i4'), ('weight', '<f8')])

In [12]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':((np.str_, 10), int, np.float32)})  # 数值数据类型可以用 Python 类型或 NumPy 的 dtype 类型指定

dtype([('name', '<U10'), ('age', '<i8'), ('weight', '<f4')])

In [13]:
# 元祖列表
np.dtype([('name', 'S10'), ('age', 'i4'), ('weight', 'f8')])

dtype([('name', 'S10'), ('age', '<i4'), ('weight', '<f8')])

In [14]:
# 如果类型的名称对你来说并不重要，那你可以仅仅用一个字符串来指定它。
np.dtype('S10,i4,f8')

dtype([('f0', 'S10'), ('f1', '<i4'), ('f2', '<f8')])

**NumPy的数据类型**

| NumPy数据类型符号 | 描述 | 示例 |
| -- | -- | -- |
| 'b' | 字节型 | np.dtype('b') |
| 'i' | 有符号整型 | np.dtype('i4') == np.int32 |
| 'u' | 无符号整型 | np.dtype('u1') == np.uint8 |
| 'f' | 浮点型 | np.dtype('f8') == np.int64 |
| 'c' | 复数浮点型 | np.dtype('c16') == np.complex128 |
| 'S'、'a' | 字符串 | np.dtype('S5') |
| 'U' | Unicode 编码字符串 | np.dtype('U') == np.str_ |
| 'V' | 原生数据，raw data(空, void) | np.dtype('V') == np.void |

## 2. 更高级的复合类型

下例中 `X` 数组的每个元素都包含一个 `id` 和一个 `3×3` 的矩阵。  
为什么我们宁愿用这种方法存储数据，也不用简单的多维数组，或者 `Python` 字典呢？  
原因是 `NumPy` 的 `dtype` 直接映射到 `C` 结构的定义，因此包含数组内容的缓存可以直接在 `C` 程序中使用。  
如果你想写一个 `Python` 接口与一个遗留的 `C` 语言或 `Fortran` 库交互，从而操作结构化数据，你将会发现结构化数组非常有用！

In [15]:
tp = np.dtype([('id', 'i8'), ('mat', 'f8', (3, 3))])
X = np.zeros(1, dtype=tp)
print(X[0])
print(X['mat'][0])

(0, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


## 3. 记录数组：结构化数组的扭转

`NumPy` 还提供了 `np.recarray` 类。它和前面介绍的结构化数组几乎相同，但是它有一个独特的特征：域可以像属性一样获取，而不是像字典的键那样获取。

In [16]:
data['age']

array([25, 45, 37, 19], dtype=int32)

In [17]:
data_rec = data.view(np.recarray)
data_rec.age  # 如果将这些数据当作一个记录数组，我们可以用很少的按键来获取这个结果：

array([25, 45, 37, 19], dtype=int32)

In [ ]:
# 记录数组的不好的地方在于，即使使用同样的语法，在获取域时也会有一些额外的开销
%timeit data['age']
%timeit data_rec['age']
%timeit data_rec.age

## 4. 关于Pandas

本章将结构化数组和记录数组放在末尾是有意为之，因为它们能很好地衔接下一章要介绍的包：`Pandas`。  
本章介绍的结构化数组在某些场景中很好用，特别是当你用 `C`、`Fortran` 或其他语言将 `NumPy` 数组映射为二进制数据格式时。  
但是如果每天都需要使用结构化数据，那么 `Pandas` 包是更好的选择，我们将在接下来的一章详细介绍它。